In [7]:
import pandas as pd
import numpy as np
#!pip install -U sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [8]:
# Класс предсказания состояний на основе RDF
class DesignForestModel():
    def __init__(self):
        self.df = pd.read_csv('lander_data.csv').head(2000)
        self.seed = 50
        self.model = self._learn_model()
        
    # Обучение модели
    def _learn_model(self):
        # Выбираем найзвания столбцов из датасета
        X = self.df[['X', 'Y', 'VelX', 'VelY', 'Angle', 'AngularVel', 'ActionID']]
        y = np.array([int(abs(self.df['ResX'][i]) * 100000) + int(abs(self.df['Angle'][i]) * 100000) 
                           for i in range(len(self.df[['ResX']]))])
        # 70% выборки идёт на обучение 
        train_X, test_X, train_y, test_y = train_test_split(X, y,
                                                 test_size = 0.3, 
                                                 random_state = self.seed)

        # Создаём модель леса из 100 деревьев
        model = RandomForestClassifier(n_estimators = 100,
                                       random_state = self.seed,
                                       max_features = 'sqrt',
                                       n_jobs = -1,
                                       verbose = 0)

        # Обучение модели на тренировочных данных 
        model.fit(train_X, train_y)
        return model
    
    # Метод предсказания состояния при воздействии на текущее состояние
    # Вход:  DataFrame из состояния + действие
    # Выход: DataFrame из предсказанных состояний (X, alpha)
    def GetNextStates(self, current_states):
        prediction = self.model.predict(current_states)
        return prediction


In [9]:
from abc import ABCMeta, abstractmethod
# Абстрактный класс подбора оптимального управления
class ASolver():
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def Solve(state):
        """ Подбор оптимального действия по текущему состоянию """
        

# Класс подбора оптимального управления
class Solver_v1(ASolver):
    def __init__(self, forestModel : DesignForestModel):
        self.forestModel = forestModel
        
    # Подбор оптимального действия по текущему состоянию
    # Вход:  состояние
    # Выход: действие
    def Solve(self, state):
        df = pd.DataFrame(index = [0, 1, 2, 3], columns = ['X', 'Y', 'VelX', 'VelY', 'Angle', 'AngylarVel', 'ActionID'])
        for i in range(4):
            for j in range(7):
                df.loc[i][j] = state[j] if j != 6 else i
        
        prediction = self.forestModel.GetNextStates(df)
        action = np.argmin([abs(prediction) for i in range(4)])
        return action
    

In [10]:
# Класс модуля управления
class ControlModule():
    def __init__(self):
        None
        
    # Метод получения управления от состояния
    # Вход:  сотояние и алгоритм решения
    # Выход: действие, полученное алгоритмом
    def GetControl(self, state, solver : ASolver):
        self.solver = solver
        action = self.solver.Solve(state)
        return action
    

In [11]:
# Клиентский код
#!pip install -U gym
import gym

# Инициализируем среду
env = gym.make('LunarLander-v2')
# Количество запусков игры
count_of_episodes = 5
# Максимальное количество итераций в запуске
count_of_steps = 200

# Дерево решений
tree_forest = DesignForestModel()
# Объект класса поиска управления
solver = Solver_v1(tree_forest)
# Объект модуля управления
control_module = ControlModule()

In [12]:
for i_episode in range(count_of_episodes):
        cur_reward = 0
        observation = env.reset()
        for step_ind in range(count_of_steps):
            env.render()
            if(step_ind % 2 == 0):
                action = control_module.GetControl(observation[0:6], solver)
            else:
                action = 0
            
            observation, reward, done, info = env.step(action)
            
            if done:
                print("Episode {} finished with reward: {}".format(i_episode, cur_reward))
                break
            else:
                cur_reward = reward
env.close()

Episode 0 finished with reward: 18.104202799707167
Episode 1 finished with reward: -26.98062618867931
Episode 2 finished with reward: 38.15045347306048
Episode 3 finished with reward: 6.526589442093325
Episode 4 finished with reward: 8.011861570816706
